# ML Pipeline Preparation


### 1. Import libraries and load data from database.


In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy.engine import create_engine

In [5]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('data', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = df['message']
Y = df.iloc[:,-36:]

In [7]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data


In [8]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#libraries for the tokenize function
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) #removing all punctuations
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        #clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
        
    return clean_tokens
    

In [10]:
#checking how tokenize() works
for text in X[:5]:
    tokens = tokenize(text)
    print(text)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
#libraries to build the ML model
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier


In [12]:
Y.shape[1] #n_classes for the multilabel classfication - labels

36

In [14]:
X.shape

(26028,)

In [13]:
pipeline = Pipeline([
            ('vect', CountVectorizer(token_pattern=None,tokenizer = tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)


In [13]:
#saving the split datasets to

In [16]:
X_train.shape


(20822,)

In [17]:
X_train

11156    I am willing to donate my time , food and othe...
24149    Reports also indicated that the power supply, ...
10183    I just looked at some of the earthquake pictur...
20665    Sufficient stocks of cultivated crops are stil...
18915    In addition to all this, there are growing fea...
                               ...                        
17985    Water and sanitation (53% funded): Improved ac...
10018    HELP THE EARTHQUAKE VICTIMS IN HAITI http tiny...
13685    Visitors over the years would notice the gradu...
15905    People in the Northern Central and Eastern are...
16663    According to local hydrologic stations, the wa...
Name: message, Length: 20822, dtype: object

In [18]:
y_train.shape

(20822, 36)

In [19]:
%%time
pipeline.fit(X_train,y_train)

CPU times: user 8min 31s, sys: 3.35 s, total: 8min 34s
Wall time: 8min 39s


Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x13acf24d0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
y_pred = pipeline.predict(X_test)

In [21]:
y_pred.shape

(5206, 36)

In [22]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [23]:
labels = y_test.values
labels.shape

(5206, 36)

In [24]:
columns = y_test.columns #true labels to np array
columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [35]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [25]:
from sklearn.metrics import classification_report

for i, col in enumerate(columns):
    print('Classification report for: ' + col)
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

Classification report for: related
              precision    recall  f1-score   support

           0       0.76      0.30      0.43      1213
           1       0.82      0.97      0.89      3993

    accuracy                           0.81      5206
   macro avg       0.79      0.64      0.66      5206
weighted avg       0.81      0.81      0.78      5206

Classification report for: request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4295
           1       0.91      0.46      0.61       911

    accuracy                           0.90      5206
   macro avg       0.90      0.72      0.77      5206
weighted avg       0.90      0.90      0.88      5206

Classification report for: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5183
           1       0.00      0.00      0.00        23

    accuracy                           1.00      5206
   macro avg       0.50  

/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      4520
           1       0.63      0.02      0.05       686

    accuracy                           0.87      5206
   macro avg       0.75      0.51      0.49      5206
weighted avg       0.84      0.87      0.81      5206

Classification report for: infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4888
           1       0.50      0.00      0.01       318

    accuracy                           0.94      5206
   macro avg       0.72      0.50      0.49      5206
weighted avg       0.91      0.94      0.91      5206

Classification report for: transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4960
           1       0.78      0.09      0.15       246

    accuracy                           0.96      5206
   macro avg       0.87      0.54      0.

/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
from sklearn.model_selection import GridSearchCV
#this is the default settings used in pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect', CountVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x13acf24d0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x13acf24d0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': None,
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth

In [27]:
from sklearn.model_selection import GridSearchCV
parameters =  {'clf__estimator__n_jobs':[2,4,6]
                   }

cv = GridSearchCV(pipeline, param_grid = parameters)


In [29]:
%%time
cv.fit(X_train, y_train)

CPU times: user 2h 14min 19s, sys: 1min 18s, total: 2h 15min 38s
Wall time: 43min 17s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(token_pattern=None,
                                                        tokenizer=<function tokenize at 0x13acf24d0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_jobs': [2, 4, 6]})

In [30]:
cv.best_params_

{'clf__estimator__n_jobs': 4}

In this pipeline, using n_jobs=6 is the best parameter

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
y_pred2 = cv.best_estimator_.predict(X_test)
for column in range(len(y_pred2.T)):
    # Print the classification report
    print(classification_report(labels[:, column], y_pred2[:, column]))

              precision    recall  f1-score   support

           0       0.74      0.30      0.43      1213
           1       0.82      0.97      0.89      3993

    accuracy                           0.81      5206
   macro avg       0.78      0.63      0.66      5206
weighted avg       0.80      0.81      0.78      5206

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4295
           1       0.90      0.46      0.61       911

    accuracy                           0.90      5206
   macro avg       0.90      0.72      0.77      5206
weighted avg       0.90      0.90      0.88      5206

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5183
           1       0.00      0.00      0.00        23

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      5206

              preci

/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4937
           1       0.88      0.08      0.14       269

    accuracy                           0.95      5206
   macro avg       0.91      0.54      0.56      5206
weighted avg       0.95      0.95      0.93      5206

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5110
           1       0.75      0.03      0.06        96

    accuracy                           0.98      5206
   macro avg       0.87      0.52      0.53      5206
weighted avg       0.98      0.98      0.97      5206

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5180
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      5206

              preci

/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/isabellevea/anaconda3/envs/DataSci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [32]:
print(cv.best_params_)

{'clf__estimator__n_jobs': 4}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF


In [33]:
#this is the default settings used in pipeline
pipeline.get_params()


{'memory': None,
 'steps': [('vect', CountVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x13acf24d0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x13acf24d0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': None,
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth

In [34]:
#fine tuning parameters for tfidf
parameters_tfidf = {'tfidf__norm': ['l1', 'l2'],
                    'tfidf__smooth_idf': [True, False],
                    'tfidf__sublinear_tf': [True, False],
                    'tfidf__use_idf': [True, False]
                   }

cv_tfidf = GridSearchCV(pipeline, param_grid = parameters_tfidf)


In [35]:
%%time
cv_tfidf.fit(X_train, y_train)


CPU times: user 7h 2min 56s, sys: 4min 45s, total: 7h 7min 42s
Wall time: 23h 1min 2s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(token_pattern=None,
                                                        tokenizer=<function tokenize at 0x13acf24d0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'tfidf__norm': ['l1', 'l2'],
                         'tfidf__smooth_idf': [True, False],
                         'tfidf__sublinear_tf': [True, False],
                         'tfidf__use_idf': [True, False]})

In [36]:
print(cv_tfidf.best_params_)

{'tfidf__norm': 'l2', 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True}


In [37]:
#new pipeline using the best parameters for tfidf
y_pred3 = cv_tfidf.best_estimator_.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score

for column in range(len(y_pred2.T)):
    # Print the classification report
    print(confusion_matrix(labels[:, column], y_pred2[:, column]))
    print(accuracy_score(labels[:, column], y_pred2[:, column]))
    print('Improvement of {:0.2f}%.'.format( 100 * 
                                            ((accuracy_score(labels[:, column], y_pred2[:, column]))-
                                             (accuracy_score(labels[:, column], y_pred[:, column])))/
                                            (accuracy_score(labels[:, column], y_pred[:, column]))))
    print()


[[ 362  851]
 [ 124 3869]]
0.8127160968113715
Improvement of -0.21%.

[[4248   47]
 [ 492  419]]
0.8964656165962351
Improvement of -0.02%.

[[5183    0]
 [  23    0]]
0.9955820207452939
Improvement of 0.00%.

[[2602  378]
 [ 781 1445]]
0.7773722627737226
Improvement of 0.02%.

[[4789   18]
 [ 377   22]]
0.9241260084517864
Improvement of -0.10%.

[[4936    5]
 [ 250   15]]
0.9510180560891279
Improvement of -0.04%.

[[5059    0]
 [ 145    2]]
0.9721475220898963
Improvement of 0.02%.

[[5124    0]
 [  82    0]]
0.9842489435267
Improvement of 0.00%.

[[5028    1]
 [ 170    7]]
0.9671532846715328
Improvement of 0.04%.

[[5206]]
1.0
Improvement of 0.00%.

[[4847    4]
 [ 278   77]]
0.9458317326162121
Improvement of -0.04%.

[[4535   38]
 [ 374  259]]
0.9208605455243949
Improvement of -0.48%.

[[4724   27]
 [ 319  136]]
0.933538225124856
Improvement of 0.31%.

[[5108    3]
 [  86    9]]
0.9829043411448329
Improvement of -0.04%.

[[5073    1]
 [ 130    2]]
0.9748367268536304
Improvement of 0.0

#### Fine tuning tf-idf part of the pipeline did not lead to subtantial accuracy improvement.


## What parameters to fine tune?

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

According to this article: https://towardsdatascience.com/random-forest-hyperparameters-and-how-to-fine-tune-them-17aee785ee0d the important parameters in RandomForestClassifier are:

- "The Nº of Decision Trees in the forest (in Scikit-learn this parameter is called n_estimators): The classification error decreases when we increase the number of trees." 
So we will test that by using 3 values including a small number and a large number.

- "The criteria with which to split on each node (Gini or Entropy for a classification task, or the MSE or MAE for regression)"

- "Number of random features to include at each node for splitting." 



In [39]:
print(pipeline.get_params().keys())

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estim

In [40]:
from sklearn.model_selection import GridSearchCV
n_estimators = [10, 100, 1000]
criterion = ['gini', 'entropy', 'log_loss']
max_features = ['sqrt', 'log2', None] # we can also look at int and float values but I decided not to to reduce the number of searches for this exercise.


parameters_clf = {'clf__estimator__n_estimators': n_estimators,
                  'clf__estimator__criterion': criterion,
                'clf__estimator__max_features': max_features}


cv_clf = GridSearchCV(pipeline, param_grid = parameters_clf)


In [1]:
%%time
cv_clf.fit(X_train, y_train)


NameError: name 'cv_clf' is not defined

In [ ]:
print(cv_clf.best_params)

### 9. Export your model as a pickle file

In [ ]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(cv_clf, file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

References I used for this notebook
https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

Model evaluation: https://medium.com/analytics-vidhya/evaluating-a-random-forest-model-9d165595ad56